In [1]:
import reaction_parallel_cython
import numpy as np

In [11]:
N = 20000

cellsize = 100

Particle_dtype = np.dtype([
    ('pos', np.float64, (3,)),
    ('vel', np.float64, (3,)),
    ('E', np.float64),
    ('id', np.int32)
], align=True)  # 添加 align=True

Cell_dtype = np.dtype([
    ('id', np.int32),
    ('index', np.int32, (3,)),
    ('film', np.int32, (5,)),
    ('normal', np.float64, (3,))
], align=True)  # 添加 align=True

particle = np.zeros(N, dtype=Particle_dtype)
# cell = np.zeros((cellsize, cellsize, cellsize), dtype=CELLDTYPE)

pos = np.random.rand(N, 3)*99
vel = np.random.rand(N, 3)
energy = np.linalg.norm(vel, axis=1)
vel[:,0] = np.divide(vel[:,0], energy)
vel[:,1] = np.divide(vel[:,1], energy)
vel[:,2] = np.divide(vel[:,2], energy)


# 填充粒子数据
particle['pos'] = pos
particle['vel'] = vel
particle['E'] = 100
particle['id'] = 0
particle[int(N/2):]['id'] = 1
cell = np.zeros((cellsize, cellsize, cellsize), dtype=Cell_dtype)

nn = np.array([0, 0, 1])
# nn = nn/np.linalg.norm(nn)

cell['id'] = 1
cell['index'] = np.array([1,2,3])
cell['film'] = np.array([10,10,10,10,0])
cell['normal'] = nn

cell[:,:,50:]['id'] = 1
print(Particle_dtype.itemsize)
print(Cell_dtype.itemsize)

64
64


In [8]:
particle

array([([16.65311943, 48.22045951, 58.68673518], [0.64476753, 0.72812721, 0.23260611], 100., 0),
       ([35.52118164, 50.79460337, 57.62163494], [0.76219828, 0.48534668, 0.42836011], 100., 0),
       ([94.92502984, 44.41277864, 34.34971596], [0.59248908, 0.67088151, 0.44595368], 100., 0),
       ...,
       ([20.94351822, 17.33556662, 60.95141199], [0.43241061, 0.76726976, 0.4736224 ], 100., 1),
       ([95.96113395, 91.68516742, 78.87142577], [0.93935201, 0.10670983, 0.32593069], 100., 1),
       ([56.9476404 , 73.05607821, 52.45452188], [0.92383959, 0.28934977, 0.25059356], 100., 1)],
      dtype={'names': ['pos', 'vel', 'E', 'id'], 'formats': [('<f8', (3,)), ('<f8', (3,)), '<f8', '<i4'], 'offsets': [0, 24, 48, 56], 'itemsize': 64, 'aligned': True})

In [12]:
a = reaction_parallel_cython.particle_parallel_vector(particle, cell, np.array([cellsize, cellsize, cellsize], dtype=np.double))

In [13]:
%timeit -n 100 reaction_parallel_cython.particle_parallel_vector(particle, cell, np.array([cellsize, cellsize, cellsize], dtype=np.double))

24.4 ms ± 397 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
np.where(a[0]==-1)



(array([], dtype=int64),)

In [1]:
particle

NameError: name 'particle' is not defined

In [8]:
np.linalg.norm(np.array([-0.19690162, -0.08145782, -0.97703346]))

1.0000000031786742

In [10]:
np.linalg.norm(np.array([0.60504164, 0.44881105, 0.6576422 ]))

1.000000003978416

In [7]:
# print(a.shape)
print(a[0])
print(a[1])
print(a[2])
print(a[3])

[ 0  0  0  0  0  0  0  0  0  0  1  2  2  1 -1  2  0 -1  2  2]
[[1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]
 [1 1 1 0 0]
 [0 1 1 0 0]
 [0 0 1 0 0]
 [0 1 1 0 0]
 [0 0 0 0 0]
 [1 1 1 0 0]
 [1 1 1 0 0]
 [0 0 0 0 0]
 [0 0 1 0 0]
 [0 1 1 0 0]]
[[0.88452079 0.         0.         0.         0.        ]
 [0.79724123 0.         0.         0.         0.        ]
 [0.7898093  0.         0.         0.         0.        ]
 [0.33746254 0.         0.         0.         0.        ]
 [0.6740954  0.         0.         0.         0.        ]
 [0.47123019 0.         0.         0.         0.        ]
 [0.5827221  0.         0.         0.         0.        ]
 [0.43777867 0.         0.         0.         0.        ]
 [0.29238065 0.         0.         0.         0.        ]
 [0.56943119 0.         0.         0.         0.        ]
 [0.26073543 0.66737785 0.39280574 0.         0.        ]
 [0.         0.39113413 0.70304003 0.  

In [8]:
print(a[4])

[[-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  1  0  0  0]
 [-1  0  0  0  0]
 [ 0 -1  0  0  0]
 [ 0  0  0  0  0]
 [ 0  0  0 -1  0]
 [ 0 -1  0  0  0]
 [ 0  0 -1  0  0]
 [ 0  0  0 -1  0]
 [ 0  0 -1  0  0]
 [-1  0  0  0  0]
 [-1  0  0  0  0]]


In [9]:
print(a[5])

[[-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]
 [-1 -1 -1]]


In [10]:
print(a[6])

[0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]


In [7]:
cell.shape

(100, 100, 100)

In [8]:
added = np.array(np.where(cell['film'][:,:,:,0] != 10)).T

In [9]:
print(added)

[[ 1  9 91]
 [ 5 84 44]
 [26 46 31]
 [27 31 88]
 [42 73 49]
 [45  1 37]
 [47 70 92]
 [61 24 86]
 [84 87 52]
 [92 20 61]
 [93 66  2]
 [95 32  7]]


In [10]:
i = 2
cell[added[i][0],added[i][1],added[i][2]]

(1, [1, 2, 3], [ 9, 11, 10, 10,  0], [0., 0., 1.])

In [14]:
a = reaction_parallel_cython.react_add_test2(particle[0])

print(a)

[0, -1, 1, 0, 0]
[0, -1, 1, 0, 0]


In [ ]:
a = reaction_parallel_cython.react_add_test3(particle)

print(a)

In [3]:
a = reaction_parallel_cython.particle_parallel(particle, cell)

In [ ]:
a.min()

In [ ]:
print(a.shape)
print(a[:200])

In [ ]:
%timeit reaction_parallel_cython.particle_parallel(particle, cell)

In [ ]:
print(a)

In [50]:
@jit(nopython=True, parallel=True)
def particle_parallel_numba(particles, cell):

    dot_product = np.zeros(particles.shape[0], dtype=np.double)

    for i in prange(particles.shape[0]):
        cellijk = particles[i]['pos'].astype(np.int32)

        if cell[cellijk[0],cellijk[1], cellijk[2]]['id'] == 1:
            dot_product[i] = np.dot(particles[i]['vel'], cell[cellijk[0],cellijk[1], cellijk[2]]['normal'])

    return dot_product

In [ ]:
np.arange(0, np.pi/2, 0.01, dtype=np.double).shape

In [ ]:
np.arange(0, np.pi/2, 0.01, dtype=np.double)

In [ ]:
np.linspace(0, np.pi/2, 180, dtype=np.double)

In [ ]:
np.linspace(0, np.pi/2, 180, dtype=np.double).shape

In [38]:

def particle_parallel(particles, cell):

    dot_product = np.zeros(particles.shape[0], dtype=np.double)

    for i in range(particles.shape[0]):
        celli = particles[i]['pos'][0].astype(np.int32)
        cellj = particles[i]['pos'][1].astype(np.int32)
        cellk = particles[i]['pos'][2].astype(np.int32)

        if cell[celli, cellj, cellk]['id'] == 1:
            dot_product[i] = np.dot(particles[i]['vel'], cell[celli, cellj, cellk]['normal'])

    return dot_product

In [ ]:
%timeit particle_parallel_numba(particle, cell)

In [ ]:
%timeit particle_parallel(particle, cell)